# Disclaimer

- Código inicial feito pelo Thiago Rodrigues.
- Modificações realizadas por João Vítor Fernandes Dias

In [1]:
""" Instalações """

%pip install --upgrade python-math
%pip install --upgrade opencv-python
%pip install --upgrade numpy

Note: you may need to restart the kernel to use updated packages.
                                              0.0/38.1 MB ? eta -:--:--
                                              0.4/38.1 MB 11.2 MB/s eta 0:00:04
                                              0.8/38.1 MB 10.8 MB/s eta 0:00:04
     -                                        1.3/38.1 MB 10.6 MB/s eta 0:00:04
     -                                        1.6/38.1 MB 9.5 MB/s eta 0:00:04
     --                                       2.1/38.1 MB 9.1 MB/s eta 0:00:04
     --                                       2.6/38.1 MB 9.2 MB/s eta 0:00:04
     ---                                      3.0/38.1 MB 9.1 MB/s eta 0:00:04
     ---                                      3.5/38.1 MB 9.3 MB/s eta 0:00:04
     ----                                     3.9/38.1 MB 9.2 MB/s eta 0:00:04
     ----                                     4.4/38.1 MB 9.6 MB/s eta 0:00:04
     ----                                     4.8/38.1 MB 9.5 MB/s et

ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\joaov\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\~-mpy\\.libs\\libopenblas64__v0.3.21-gcc_10_3_0.dll'
Check the permissions.



In [2]:
""" Importação de bibliotecas + Constantes """

import math
import cv2  # pip install opencv-python
import numpy as np  # pip install numpy
# from matplotlib import pyplot as plt
# from numpy.lib.function_base import angle

BASE_PATH = 'Files/Code/Filtros e Comparar Assinatura/'
FOLDER = 'img/'
FILE_NAMES = [
    'assinatura_internet1.png',
    'assinatura_vermelha.png',
    'assinatura_preta.png',
]
paths = []
for item in FILE_NAMES:
    paths.append(FOLDER + item)

In [3]:
""" Funções para mostrar imagens """

def print_image(image, title = 'Image'):
    # Display image
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def print_image_path(path):
    img = cv2.imread(path)
    print_image(img)

In [4]:
# Debug - Erasable

# print_image(image1)
# print_image(image2)

# print_image_path(paths[0])
# print_image_path(paths[1])

In [5]:
""" Função proc. retorna uma lista contendo os seguintes dados:
    [
        image, grayImg, imMedia, imGaussian, imFourier, imSobel, imLaplace,
        binSobel, binLaplace, binOriginal,
        imgLines, imgR1, intpts, cms
    ]
"""

def proc(image):
    """ Processamento [de algo] """

    def MeanFilter(grayImg):
        """ Filtro de Média """
        
        # Definindo a dimensão que os lados do filtro terão
        filter_dim = 3
        den = filter_dim*filter_dim

        # Cria uma matriz de 0s com as dimensões definidas anteriormente
        filter_Media = np.zeros((filter_dim, filter_dim), np.float32)

        # Cálculo do filtro
        for x in range(filter_dim):
            for y in range(filter_dim):
                filter_Media[x, y] = 1/den

        imgFiltrada = np.zeros_like(grayImg, dtype=np.float32)
        imgFiltrada = convolution(grayImg, filter_Media, filter_dim)
        return (imgFiltrada.astype(np.uint8))

    def GaussianFilter(grayImg, sigma):
        """ Filtro Gaussiano """

        # Definindo a dimensão que os lados do filtro terão
        filter_dim = 3
        imgFiltrada = cv2.GaussianBlur(grayImg, (3, 3), sigma)

        # Cria uma matriz de 0s com as dimensões definidas anteriormente
        #     filter_Gauss = np.zeros((filter_dim, filter_dim), np.float32)
        #     d = filter_dim//2

        #     #Cálculo do filtro
        #     for x in range(-d, d+1):
        #         for y in range(-d, d+1):
        #             filter_Gauss[x+d, y+d] = (1/(2*np.pi*(sigma**2)))*(np.exp(-(x**2 + y**2)/(2* sigma**2)))

        #     print (filter_Gauss)
        #     imgFiltrada = np.zeros_like(grayImg, dtype=np.float32)

        #     imgFiltrada = convolution(grayImg, filter_Gauss, filter_dim)

        return (imgFiltrada.astype(np.uint8))

    def FourierFilter(grayImg, r):
        """ Filtro de Fourier """

        # Fourier transforma a imagem, fft é uma matriz tridimensional, fft [:,:, 0] é a parte real, fft [:,:, 1] é a parte imaginária
        fft = cv2.dft(np.float32(grayImg), flags=cv2.DFT_COMPLEX_OUTPUT)

        # Centraliza fft, o dshift gerado ainda é uma matriz tridimensional
        dshift = np.fft.fftshift(fft)

        # Obtenção do pixel central
        rows, cols = grayImg.shape[:2]
        mid_row, mid_col = int(rows / 2), int(cols / 2)

        # Construção da máscara
        mask = np.zeros((rows, cols, 2), np.float32)
        mask[mid_row - r:mid_row + r, mid_col - r:mid_col + r] = 1
        fft_filtering = dshift * mask

        # Transformada inversa
        ishift = np.fft.ifftshift(fft_filtering)
        image_filtering = cv2.idft(ishift)
        image_filtering = cv2.magnitude(image_filtering[:, :, 0], image_filtering[:, :, 1])

        # Normaliza os resultados da transformação inversa
        cv2.normalize(image_filtering, image_filtering, 0, 1, cv2.NORM_MINMAX)
        return image_filtering

    def convolution(original, kernel, kernel_dim):
        """ Função de Convolução """

        kd = kernel_dim // 2
        image_pad = np.pad(original, pad_width=((kd, kd), (kd, kd)), mode='constant', constant_values=0).astype(np.float32)
        image_conv = np.zeros(image_pad.shape)

        for i in range(kd, image_pad.shape[0]-kd):
            for j in range(kd, image_pad.shape[1]-kd):
                x = image_pad[i-kd:i-kd+kernel_dim, j-kd:j-kd+kernel_dim]
                x = x.flatten()*kernel.flatten()
                image_conv[i][j] = x.sum()
 
        return image_conv

    def get_gray_value(image, i, j):
        """ Cálculo do valor de cinza a partir da formula Cinza = Azul*0.07 + Verde*0.72 + Vermelho*0.21 """
        return 0.07 * image[i, j, 0] + 0.72 * image[i, j, 1] + 0.21 * image[i, j, 2]

    def SobelFilter(image):
        """ Filtro de Sobel """

        matrix_a = [
            [-1, 0, 1],
            [-2, 0, 2],
            [-1, 0, 1]
        ]

        matrix_b = [
            [ 2,  2,  4,  2,  2],
            [ 1,  1,  2,  1,  1],
            [ 0,  0,  0,  0,  0],
            [-1, -1, -2, -1, -1],
            [-2, -2, -4, -2, -2]
        ]

        matrices = [matrix_a, matrix_b]
        filter = np.array(matrices[1])
        filter_dim = len(matrices[1])

        imSV = convolution(image, filter, filter_dim)
        imSH = convolution(image, filter.T, filter_dim)

        imgFiltrada = np.zeros_like(image, dtype=np.float32)
        d = np.zeros_like(image, dtype=np.float32)

        for i in range(H):
            for j in range(W):
                imgFiltrada[i, j] = image[i, j]
                d[i, j] = np.hypot(imSV[i, j], imSH[i, j])
                if d[i, j] > 90.0:
                    imgFiltrada[i, j] = image[i, j]*0.7

        return imgFiltrada.astype(np.uint8), filter_dim

    def LaplaceFilter(image):
        """ Filtro Laplaciano """
        laplace_matrix_1 = [
            [ 0, -1,  0],
            [-1,  4, -1],
            [ 0, -1,  0],
        ]
        laplace_matrix_2 = [
            [ 1, -2,  1],
            [-2,  4, -2],
            [ 1, -2,  1],
        ]
        laplace_matrix_3 = [
            [-1, -1, -1],
            [-1,  8, -1],
            [-1, -1, -1],
        ]
        matrices = [laplace_matrix_1, laplace_matrix_2, laplace_matrix_3]
        filtro = np.array(matrices[1])

        filter_dim = 3
        img = convolution(image, filtro, filter_dim)
        imgFiltrada = np.zeros_like(image, dtype=np.float32)

        for i in range(H):
            for j in range(W):
                imgFiltrada[i, j] = 255 + 0 if image[i, j] > 245 else -img[i, j]

        return imgFiltrada.astype(np.uint8), filter_dim

    def binaryImage(image, kd): # Upgrade later
        """ Binarização com Otsu """
        
        histog_val, bordas_col = np.histogram(image, 256)       # Cálculo do histograma e das bordas das colunas
        centros_col = (bordas_col[:-1] + bordas_col[1:]) / 2.   # Cálculo dos centros de cada coluna
        peso_cresc = np.cumsum(histog_val)          # Cálculo do peso em cada coluna de forma cumulativa da Esquerda para a Direita
        peso_decresc = np.cumsum(histog_val[::-1])[::-1]        # Cálculo do peso em cada coluna de forma cumulativa da Direita para a Esquerda

        # Cálculo das médias
        media_cresc = np.cumsum(histog_val * centros_col) / peso_cresc
        media_decresc = (np.cumsum((histog_val * centros_col)[::-1]) / peso_decresc[::-1])[::-1]

        # Cálculo das variâncias interclasses
        variancia_interclasses = peso_cresc[:-1] * peso_decresc[1:] * (media_cresc[:-1] - media_decresc[1:]) ** 2

        # Cálculo do caso que tem a variância máxima
        max_variancia = np.argmax(variancia_interclasses)

        # Definição do melhor linear
        linear = centros_col[:-1][max_variancia]
        # print("\nLimiar ", linear)

        # Binarização
        binImg = np.zeros((H, W), np.uint8)
        for i in range(H):
            for j in range(W):
                binImg[i, j] = 0 if image[i, j] < linear else 255

        jMin = jMax = iMin = iMax = 0
        
        # Encontrando valor mínimo de j
        for i in range(kd, H):
            for j in range(kd, W):
                if binImg[i, j] == 0:
                    if jMin == 0:
                        jMin = j
                    elif j < jMin:
                        jMin = j

        # Encontrando valor máximo de j
        for i in range(kd, H):
            for j in range(kd, W):
                if binImg[i, j] == 0:
                    if jMax == 0:
                        jMax = j
                    elif j > jMax:
                        jMax = j

        # Encontrando valor mínimo de i
        for i in range(kd, H):
            for j in range(kd, W):
                if binImg[i, j] == 0:
                    if iMin == 0:
                        iMin = i
                    elif i < iMin:
                        iMin = i

        # Encontrando valor máximo de i
        for i in range(kd, H):
            for j in range(kd, W):
                if binImg[i, j] == 0 and (iMax == 0 or i > iMax):
                    iMax = i
        
        """ Code from Phind: """
        
        # iMax = kd-1
        # i_list = [i for i in range(kd, H) if binImg[i, j] == 0]
        # if i_list:
        #     iMax = max(i_list)
        
        """ Code from Phind """

        # iMin = jMin =  float('inf')
        # iMax = jMax = -float('inf')
        # for i in range(kd, H):
        #     for j in range(kd, W):
        #         if binImg[i, j] == 0:
        #             if j < jMin:
        #                 jMin = j
        #             if j > jMax:
        #                 jMax = j
        #             if i < iMin:
        #                 iMin = i
        #             if i > iMax:
        #                 iMax = i

        # print(iMin, iMax)
        # print(jMin, jMax)

        binImgFinal = binImg[(iMin-5):(iMax+5), (jMin-5):(jMax+5)]

        return binImgFinal

    def cMDiv(image, iLmin, iLmax, jLmin, jLmax, lim):
        """ Cálculo recursivo de centros de massa """
        acumI = acumJ = contP = 0

        imgCopia = np.zeros_like(image, dtype=np.float32)
        imgLines = np.zeros_like(image, dtype=np.float32)

        for i in range(iLmin, iLmax):
            for j in range(jLmin, jLmax):
                imgCopia[i, j] = image[i, j]
                if image[i, j] == 0:
                    contP += 1
                    acumI += i
                    acumJ += j

        if lim <= 2 and contP != 0:
            cmI = acumI//contP
            cmJ = acumJ//contP
            cms.append((cmI, cmJ))

            lim += 1

            imgLines = cMDiv(imgCopia, iLmin, cmI, jLmin, cmJ, lim)
            imgLines = cMDiv(imgCopia, iLmin, cmI, cmJ, jLmax, lim)
            # cv2.imshow("Imagem CM ", imgCopia)
            imgLines = cMDiv(imgCopia, cmI, iLmax, jLmin, cmJ, lim)
            # cv2.imshow("Imagem CM ", imgCopia)
            imgLines = cMDiv(imgCopia, cmI, iLmax, cmJ, jLmax, lim)
            # cv2.imshow("Imagem CM ", imgCopia)

            # imgLines =  cv2.line(imgCopia,(jLmin,cmI),(jLmax,cmI),(0,0,255),1)
            # imgLines =  cv2.line(imgCopia,(cmJ,iLmin),(cmJ, iLmax),(0,0,255),1)
        return imgLines

    def imgRad(x, y, length, angs):
        """ Calcular raios na imagem """
        endx = []
        endy = []
        for a in range(len(angs)):
            angle = angs[a]

            yf = y + length * math.sin(math.radians(360-angle))
            xf = x + length * math.cos(math.radians(angle))

            endx.append(int(xf))
            endy.append(int(yf))

        return endx, endy

    def ptRad(angs, cms, binLaplace):
        """ Calcula interseções das letras com os raios """
        # print(binLaplace.shape)
        imgV = np.zeros_like(binLaplace, dtype=np.float32)
        imgH = np.zeros_like(binLaplace, dtype=np.float32)
        
        for i in range(binLaplace.shape[0]):
            for j in range(binLaplace.shape[1]):
                imgV[i, j] = cms[0][0] - i
                imgH[i, j] = j - cms[0][1]

        intersec = []
        # print(imgV)
        # print(imgH)
        pti = (0, binLaplace.shape[1]-cms[0][1]-1)
        # print(pti)
        npt = np.hypot(pti[0], pti[1])
        # print(npt)
        for i in range(binLaplace.shape[0]):
            for j in range(binLaplace.shape[1]):
                if binLaplace[i, j] == 0:
                    npf = np.hypot(imgV[i, j], imgH[i, j])
                    prI = (pti[0]*imgV[i, j])+(pti[1]*imgH[i, j])
                    angle = math.degrees(math.acos(prI/(npt*npf)))
                    for a in range(len(angs)):
                        if int(angle) == angs[a]:
                            value_to_append = angs[a] if imgV[i, j] >= 0 else 360-angs[a]
                            intersec.append((i, j, value_to_append))

        return intersec

    # Imagem 1
    H = image.shape[0] # Altura
    W = image.shape[1] # Largura

    # Criação de matriz com mesmo tamanho da imagem tendo todos os elementos zerados
    grayImg = np.zeros((H, W), np.uint8)

    # Cálculo do valor de cinza a partir da formula Cinza = Azul*0.07 + Verde*0.72 + Vermelho*0.21
    for i in range(H):
        for j in range(W):
            grayImg[i, j] = np.clip(get_gray_value(image, i, j), 0, 255)


    # Gerando novas imagens após a cinza original passar pelos Filtros Passa Baixa
    imFourier = FourierFilter(grayImg, 100)
    imGaussian = GaussianFilter(grayImg, 1)
    imMedia = MeanFilter(grayImg)

    # Gerando novas imagens aplicando Filtros Passa Alta na imagem suavizada anteriormente
    imSobel, dimS = SobelFilter(imMedia)
    imLaplace, dimL = LaplaceFilter(imMedia)

    # Gerando novas imagens a partir do uso da Binarização por Otsu nas imagens geradas pelos filtros Passa Alta
    binSobel = binaryImage(imSobel, (2*dimS))
    binLaplace = binaryImage(imLaplace, dimL)
    binOriginal = binaryImage(grayImg, 0)

    # print("B:", binLaplace.shape)

    iLmin = jLmin = lim = 0
    iLmax = binLaplace.shape[0]
    jLmax = binLaplace.shape[1]
    cms = []

    imCMs = cMDiv(binLaplace, iLmin, iLmax, jLmin, jLmax, lim)

    imgCMS = np.zeros_like(binLaplace, dtype=np.float32)
    imgR = np.zeros_like(binLaplace, dtype=np.float32)
    for i in range(binLaplace.shape[0]):
        for j in range(binLaplace.shape[1]):
            imgCMS[i, j] = imgR[i, j] = binLaplace[i, j]

    # Desenha pontos nos centros de massa
    for x in range(len(cms)):
        imgLines = cv2.line(imgCMS, (cms[x][1], cms[x][0]), (cms[x][1], cms[x][0]), (0, 0, 255), 6)

    # Ângulos dos raios
    angs = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]

    endx, endy = imgRad(cms[0][1], cms[0][0], 2*W, angs)

    # Desenha o centro de massa da imagem total
    imgR1 = cv2.line(imgR, (cms[0][1], cms[0][0]), (cms[0][1], cms[0][0]), (0, 0, 255), 6)

    # Desenha os raios na imagem
    for a in range(len(endx)):
        imgR1 = cv2.line(imgR, (cms[0][1], cms[0][0]),
                         (endx[a], endy[a]), (0, 0, 255), 1)

    intersec1 = ptRad(angs, cms, binLaplace)

    # Desenha pontos nos pixels pretos que fazem interseção com algum dos raios
    for p in range(len(intersec1)):
        imgR1 = cv2.line(imgR, (intersec1[p][1], intersec1[p][0]),
                         (intersec1[p][1], intersec1[p][0]), (0, 0, 255), 6)

    count = 0
    intpts = []

    for a in range(len(angs)):
        count = 0
        for q in range(len(intersec1)):
            if intersec1[q][2] == angs[a]:
                count += 1
        intpts.append(count)

    # print(intpts)

    return [image, grayImg, imMedia, imGaussian, imFourier, imSobel, imLaplace, binSobel, binLaplace, binOriginal, imgLines, imgR1, intpts, cms]


In [6]:
""" Função proc original
    ]
"""

def proc_orig(image):

    # Imagem 1
    # Altura
    H = image.shape[0]
    # Largura
    W = image.shape[1]

    # Criação de matriz com mesmo tamanho da imagem tendo todos os elementos zerados
    grayImg = np.zeros((H, W), np.uint8)
    # Cálculo do valor de cinza a partir da formula Cinza = Azul*0.07 + Verde*0.72 + Vermelho*0.21
    for i in range(H):
        for j in range(W):
            grayImg[i, j] = np.clip(
                0.07 * image[i, j, 0] + 0.72 * image[i, j, 1] + 0.21 * image[i, j, 2], 0, 255)

    ############################ FILTRO DE MEDIA ############################

    def MediaFilter(grayImg):

        # Definindo a dimensão que os lados do filtro terão
        filter_dim = 3
        den = filter_dim*filter_dim

        # Cria uma matriz de 0s com as dimensões definidas anteriormente
        filter_Media = np.zeros((filter_dim, filter_dim), np.float32)

        # Cálculo do filtro
        for x in range(filter_dim):
            for y in range(filter_dim):
                filter_Media[x, y] = 1/den

        imgFiltrada = np.zeros_like(grayImg, dtype=np.float32)

        imgFiltrada = convolution(grayImg, filter_Media, filter_dim)

        return (imgFiltrada.astype(np.uint8))

    ############################ FILTRO GAUSSIANO ############################

    def GaussianFilter(grayImg, sigma):

        # Definindo a dimensão que os lados do filtro terão
        filter_dim = 3
        imgFiltrada = cv2.GaussianBlur(grayImg, (3, 3), sigma)

        # Cria uma matriz de 0s com as dimensões definidas anteriormente
        #     filter_Gauss = np.zeros((filter_dim, filter_dim), np.float32)
        #     d = filter_dim//2

        #     #Cálculo do filtro
        #     for x in range(-d, d+1):
        #         for y in range(-d, d+1):
        #             filter_Gauss[x+d, y+d] = (1/(2*np.pi*(sigma**2)))*(np.exp(-(x**2 + y**2)/(2* sigma**2)))

        #     print (filter_Gauss)
        #     imgFiltrada = np.zeros_like(grayImg, dtype=np.float32)

        #     imgFiltrada = convolution(grayImg, filter_Gauss, filter_dim)

        return (imgFiltrada.astype(np.uint8))

    ############################ FILTRO DE FOURIER ############################

    def FourierFilter(grayImg, r):

        # Fourier transforma a imagem, fft é uma matriz tridimensional, fft [:,:, 0] é a parte real, fft [:,:, 1] é a parte imaginária
        fft = cv2.dft(np.float32(grayImg), flags=cv2.DFT_COMPLEX_OUTPUT)
        # Centraliza fft, o dshift gerado ainda é uma matriz tridimensional
        dshift = np.fft.fftshift(fft)

        # Obtenção do pixel central
        rows, cols = grayImg.shape[:2]
        mid_row, mid_col = int(rows / 2), int(cols / 2)

        # Construção da máscara
        mask = np.zeros((rows, cols, 2), np.float32)
        mask[mid_row - r:mid_row + r, mid_col - r:mid_col + r] = 1

        fft_filtering = dshift * mask
        # Transformada inversa
        ishift = np.fft.ifftshift(fft_filtering)
        image_filtering = cv2.idft(ishift)
        image_filtering = cv2.magnitude(
            image_filtering[:, :, 0], image_filtering[:, :, 1])
        # Normaliza os resultados da transformação inversa
        cv2.normalize(image_filtering, image_filtering, 0, 1, cv2.NORM_MINMAX)
        return image_filtering

    ############################ FUNÇÃO DE CONVOLUÇÃO ############################

    def convolution(original, kernel, kernel_dim):

        image_pad = np.pad(original, pad_width=((kernel_dim // 2, kernel_dim // 2), (kernel_dim //
                           2, kernel_dim // 2)), mode='constant', constant_values=0).astype(np.float32)

        kd = kernel_dim // 2

        image_conv = np.zeros(image_pad.shape)

        for i in range(kd, image_pad.shape[0]-kd):
            for j in range(kd, image_pad.shape[1]-kd):
                x = image_pad[i-kd:i-kd+kernel_dim, j-kd:j-kd+kernel_dim]
                x = x.flatten()*kernel.flatten()

                image_conv[i][j] = x.sum()
        return image_conv

    # Gerando novas imagens após a cinza original passar pelos Filtros Passa Baixa
    imGaussian = GaussianFilter(grayImg, 1)
    imMedia = MediaFilter(grayImg)
    imFourier = FourierFilter(grayImg, 100)

    ##################### FILTRO DE SOBEL ######################

    def SobelFilter(image):
        # filter = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        # filter_dim = 3

        filter = np.array([[2, 2, 4, 2, 2], [1, 1, 2, 1, 1], [
                          0, 0, 0, 0, 0], [-1, -1, -2, -1, -1], [-2, -2, -4, -2, -2]])
        filter_dim = 5

        kd = filter_dim//2

        imSV = convolution(image, filter, filter_dim)
        imSH = convolution(image, filter.T, filter_dim)

        imgFiltrada = np.zeros_like(image, dtype=np.float32)
        d = np.zeros_like(image, dtype=np.float32)

        for i in range(H):
            for j in range(W):
                imgFiltrada[i, j] = image[i, j]
                d[i, j] = np.hypot(imSV[i, j], imSH[i, j])
                if d[i, j] > 90.0:
                    imgFiltrada[i, j] = image[i, j]*0.7

        return imgFiltrada.astype(np.uint8), filter_dim

    ##################### FILTRO LAPLACIANO ######################

    def LaplaceFilter(image):

        # filter = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])
        # filter = np.array([[1, -2, 1], [-2, 4, -2], [1, -2, 1]])
        filter = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
        filter_dim = 3

        img = convolution(image, filter, filter_dim)

        imgFiltrada = np.zeros_like(image, dtype=np.float32)

        for i in range(H):
            for j in range(W):
                if image[i, j] > 245:
                    imgFiltrada[i, j] = 255
                else:
                    imgFiltrada[i, j] = 255 - img[i, j]

        return imgFiltrada.astype(np.uint8), filter_dim

    # Gerando novas imagens aplicando Filtros Passa Alta na imagem suavizada anteriormente
    imSobel, dimS = SobelFilter(imMedia)
    imLaplace, dimL = LaplaceFilter(imMedia)

    ############################## BINARIZAÇÃO COM OTSU ################################
    def binaryImage(image, kd):

        # Cálculo do histograma e das bordas das colunas
        histog_val, bordas_col = np.histogram(image, 256)

        # Cálculo dos centros de cada coluna
        centros_col = (bordas_col[:-1] + bordas_col[1:]) / 2.

        # Cálculo do peso em cada coluna de forma cumulativa da Esquerda para a Direita
        peso_cresc = np.cumsum(histog_val)

        # Cálculo do peso em cada coluna de forma cumulativa da Direita para a Esquerda
        peso_decresc = np.cumsum(histog_val[::-1])[::-1]

        # Cálculo das médias
        media_cresc = np.cumsum(histog_val * centros_col) / peso_cresc
        media_decresc = (np.cumsum((histog_val * centros_col)
                         [::-1]) / peso_decresc[::-1])[::-1]

        # Cálculo das variâncias interclasses
        variancia_interclasses = peso_cresc[:-1] * peso_decresc[1:] * (
            media_cresc[:-1] - media_decresc[1:]) ** 2

        # Cálculo do caso que tem a variância máxima
        max_variancia = np.argmax(variancia_interclasses)

        # Definição do melhor linear
        linear = centros_col[:-1][max_variancia]
        # print("\nLimiar ", linear)

        # Binarização
        binImg = np.zeros((H, W), np.uint8)
        for i in range(H):
            for j in range(W):
                if image[i, j] < linear:
                    binImg[i, j] = 0
                else:
                    binImg[i, j] = 255

        jMin = 0
        jMax = 0
        iMin = 0
        iMax = 0

        # Encontrando valor mínimo de j
        for i in range(kd, H):
            for j in range(kd, W):
                if binImg[i, j] == 0:
                    if jMin == 0:
                        jMin = j
                    else:
                        if j < jMin:
                            jMin = j

        # Encontrando valor máximo de j
        for i in range(kd, H):
            for j in range(kd, W):
                if binImg[i, j] == 0:
                    if jMax == 0:
                        jMax = j
                    else:
                        if j > jMax:
                            jMax = j

        # Encontrando valor mínimo de i
        for i in range(kd, H):
            for j in range(kd, W):
                if binImg[i, j] == 0:
                    if iMin == 0:
                        iMin = i
                    else:
                        if i < iMin:
                            iMin = i

        # Encontrando valor máximo de i
        for i in range(kd, H):
            for j in range(kd, W):
                if binImg[i, j] == 0:
                    if iMax == 0:
                        iMax = i
                    else:
                        if i > iMax:
                            iMax = i
        # print(iMin, iMax)
        # print(jMin, jMax)

        binImgFinal = binImg[(iMin-5):(iMax+5), (jMin-5):(jMax+5)]

        return binImgFinal

    # Gerando novas imagens a partir do uso da Binarização por Otsu nas imagens geradas pelos filtros Passa Alta
    binSobel = binaryImage(imSobel, (2*dimS))
    binLaplace = binaryImage(imLaplace, dimL)
    binOriginal = binaryImage(grayImg, 0)

    print("B:", binLaplace.shape)

    iLmin = 0
    iLmax = binLaplace.shape[0]
    jLmin = 0
    jLmax = binLaplace.shape[1]

    lim = 0
    cms = []

    # Cálculo recursivo de centros de massa
    def cMDiv(image, iLmin, iLmax, jLmin, jLmax, lim):

        acumI = 0
        acumJ = 0
        contP = 0

        imgCopia = np.zeros_like(image, dtype=np.float32)
        imgLines = np.zeros_like(image, dtype=np.float32)

        for i in range(iLmin, iLmax):
            for j in range(jLmin, jLmax):
                imgCopia[i, j] = image[i, j]
                if image[i, j] == 0:
                    contP += 1
                    acumI += i
                    acumJ += j

        if ((lim <= 2) and (contP != 0)):
            cmI = acumI//contP
            cmJ = acumJ//contP
            cms.append((cmI, cmJ))

            lim += 1

            imgLines = cMDiv(imgCopia, iLmin, cmI, jLmin, cmJ, lim)

            imgLines = cMDiv(imgCopia, iLmin, cmI, cmJ, jLmax, lim)
            # cv2.imshow("Imagem CM ", imgCopia)
            imgLines = cMDiv(imgCopia, cmI, iLmax, jLmin, cmJ, lim)
            # cv2.imshow("Imagem CM ", imgCopia)
            imgLines = cMDiv(imgCopia, cmI, iLmax, cmJ, jLmax, lim)
            # cv2.imshow("Imagem CM ", imgCopia)

            # imgLines =  cv2.line(imgCopia,(jLmin,cmI),(jLmax,cmI),(0,0,255),1)
            # imgLines =  cv2.line(imgCopia,(cmJ,iLmin),(cmJ, iLmax),(0,0,255),1)

            return imgLines
        else:
            return imgLines

    imCMs = cMDiv(binLaplace, iLmin, iLmax, jLmin, jLmax, lim)

    # Calcular raios na imagem
    def imgRad(x, y, length, angs):
        endx = []
        endy = []
        for a in range(len(angs)):
            angle = angs[a]

            yf = y + length * math.sin(math.radians(360-angle))
            xf = x + length * math.cos(math.radians(angle))

            endx.append(int(xf))
            endy.append(int(yf))

        return endx, endy

    # Calcula interseções das letras com os raios
    def ptRad(angs, cms, binLaplace):
        # print(binLaplace.shape)
        imgV = np.zeros_like(binLaplace, dtype=np.float32)
        imgH = np.zeros_like(binLaplace, dtype=np.float32)
        for i in range(binLaplace.shape[0]):
            for j in range(binLaplace.shape[1]):
                imgV[i, j] = cms[0][0] - i
                imgH[i, j] = j - cms[0][1]

        intersec = []
        # print(imgV)
        # print(imgH)
        pti = (0, binLaplace.shape[1]-cms[0][1]-1)
        # print(pti)
        npt = np.hypot(pti[0], pti[1])
        # print(npt)
        for i in range(binLaplace.shape[0]):
            for j in range(binLaplace.shape[1]):
                if binLaplace[i, j] == 0:
                    npf = np.hypot(imgV[i, j], imgH[i, j])
                    prI = (pti[0]*imgV[i, j])+(pti[1]*imgH[i, j])
                    angle = math.degrees(math.acos(prI/(npt*npf)))
                    for a in range(len(angs)):
                        if int(angle) == angs[a]:
                            if imgV[i, j] >= 0:
                                intersec.append((i, j, angs[a]))
                            else:
                                intersec.append((i, j, 360-angs[a]))

        return intersec

    imgCMS = np.zeros_like(binLaplace, dtype=np.float32)
    imgR = np.zeros_like(binLaplace, dtype=np.float32)
    for i in range(binLaplace.shape[0]):
        for j in range(binLaplace.shape[1]):
            imgCMS[i, j] = binLaplace[i, j]
            imgR[i, j] = binLaplace[i, j]

    # Desenha pontos nos centros de massa
    for x in range(len(cms)):
        imgLines = cv2.line(
            imgCMS, (cms[x][1], cms[x][0]), (cms[x][1], cms[x][0]), (0, 0, 255), 6)

    # Ângulos dos raios
    angs = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]

    endx, endy = imgRad(cms[0][1], cms[0][0], 2*W, angs)

    # Desenha o centro de massa da imagem total
    imgR1 = cv2.line(imgR, (cms[0][1], cms[0][0]),
                     (cms[0][1], cms[0][0]), (0, 0, 255), 6)

    # Desenha os raios na imagem
    for a in range(len(endx)):
        imgR1 = cv2.line(imgR, (cms[0][1], cms[0][0]),
                         (endx[a], endy[a]), (0, 0, 255), 1)

    intersec1 = ptRad(angs, cms, binLaplace)

    # Desenha pontos nos pixels pretos que fazem interseção com algum dos raios
    for p in range(len(intersec1)):
        imgR1 = cv2.line(imgR, (intersec1[p][1], intersec1[p][0]),
                         (intersec1[p][1], intersec1[p][0]), (0, 0, 255), 6)

    count = 0
    intpts = []

    for a in range(len(angs)):
        count = 0
        for q in range(len(intersec1)):
            if intersec1[q][2] == angs[a]:
                count += 1
        intpts.append(count)

    print(intpts)

    return [image, grayImg, imMedia, imGaussian, imFourier, imSobel, imLaplace, binSobel, binLaplace, binOriginal, imgLines, imgR1, intpts, cms]


In [7]:
""" Função Comparar """

def comparar(binLaplace1, binLaplace2, intpts1, intpts2, cms1, cms2):

    def intpM():
        intpMedia = []
        for v in range(len(intpts1)):
            x = intpts1[v] - intpts2[v]
            intpMedia.append(abs(x))
        return intpMedia

    def compativelIP():
        c = 0
        for v in range(len(intpMedia)):
            if intpMedia[v] > 65:
                return False
        return True

    def dist(cms):
        for r in range(len(cms)):
            x = 0 # Isso deveria estar fora do loop?
            x += math.sqrt((cms[0][0]-cms[r][0])**2 + (cms[0][1]-cms[r][1])**2)
        return x

    def compativelDist(distA, distB):
        size1 = binLaplace1.shape[0]*binLaplace1.shape[1]
        size2 = binLaplace2.shape[0]*binLaplace2.shape[1]

        v1 = distA/size1
        v2 = distB/size2

        return abs(v1-v2) <= 0.001

    intpMedia = intpM()
    compatibleIP = compativelIP()
    compatibleDist = compativelDist(dist(cms1), dist(cms2))
    
    print(intpMedia)

    msg = ''
    msg += 'É' if compatibleIP else 'Não é'
    msg += ' compatível pelo método das interseções\n\n'
    msg += 'É' if compatibleDist else 'Não é'
    msg += ' compatível pelo método das distâncias'
    print(msg)

In [8]:
""" Obtenção das imagens """
image1 = cv2.imread(paths[0])
image2 = cv2.imread(paths[1])
image3 = cv2.imread(paths[2])


In [9]:
""" Executando a função proc nas imagens """

data_img_1 = proc(image1)
print('processed img 1')
data_img_2 = proc(image2)
print('processed img 2')
data_img_3 = proc(image3)
print('processed img 3')


processed img 1
processed img 2
processed img 3


In [10]:
""" Executando Comparações """

binLaplace1 = data_img_1[8]
binLaplace2 = data_img_2[8]
binLaplace3 = data_img_3[8]
intpts1 = data_img_1[12]
intpts2 = data_img_2[12]
intpts3 = data_img_3[12]
cms1 = data_img_1[13]
cms2 = data_img_2[13]
cms3 = data_img_3[13]

print("\n Comparação da Imagem 2 com a Imagem 1")
comparar(binLaplace1, binLaplace2, intpts1, intpts2, cms1, cms2)
print("\n Comparação da Imagem 3 com a Imagem 1")
comparar(binLaplace1, binLaplace3, intpts1, intpts3, cms1, cms3)


 Comparação da Imagem 2 com a Imagem 1
[38, 27, 0, 20, 6, 0, 53, 15, 9, 5, 11, 10]
É compatível pelo método das interseções

Não é compatível pelo método das distâncias

 Comparação da Imagem 3 com a Imagem 1
[102, 27, 0, 20, 10, 17, 14, 24, 10, 4, 0, 13]
Não é compatível pelo método das interseções

Não é compatível pelo método das distâncias


In [11]:
""" Executando mais alguma coisa """

"""
    [
        image, grayImg, imMedia, imGaussian, imFourier, imSobel, imLaplace,
        binSobel, binLaplace, binOriginal,
        imgLines, imgR1, intpts, cms
    ]
"""
titles = [
    'Imagem Colorida',
    'Imagem Cinza',
    'Imagem Filtrada com Media',
    'Imagem Filtrada com Gaussiana',
    'Imagem Filtrada com Fourier',
    'Imagem Filtrada com Sobel',
    'Imagem Filtrada com Laplaciano',
    'Imagem Binarizada (Sobel)',
    'Imagem Binarizada (Laplaciano)',
    'Imagem Cinza Original Binarizada',
    'Imagem CM',
    'Imagem com Raios',
]


for i, title in enumerate(titles):
    print_image(data_img_1[i], title + ' 1')
    print_image(data_img_2[i], title + ' 2')
    print_image(data_img_3[i], title + ' 3')
